# Image Classification on Sewer-ML dataset

## Check Dataset Balance

In [ ]:
from TrainerHuggingFace import DataSetHF

dataset_name = "../../DeepSewerData/DeepSewer/BinaryDefectSewer-ML"
is_data_local = True
sewerMlDataSet = DataSetHF(dataset_name,is_data_local)
sewerMlDataSet.plotDataBalance()
                            

## Training with VIT

In [ ]:
from TrainerHuggingFace import TrainerHF

model_checkpoint ='google/vit-base-patch16-224'
#model_checkpoint ='microsoft/resnet-50'
#model_checkpoint ='microsoft/cvt-13'
#model_checkpoint ="nvidia/mit-b2"

dataset_name = "../../DeepSewerData/DeepSewer/imagefolder_OB_FS_OK_GR"
is_data_local = True
outName='testCustomFn'
n_epochs= 10
lr = 2e-5 
batch_size= 32
example_trainer = TrainerHF(model_checkpoint,dataset_name,is_data_local)
example_trainer.trainModel(outName,n_epochs,lr,batch_size)


# Infere with pipeline (load model and dataset)

In [ ]:
#Load Testing Dataset from class and try to print some examples
from custom_dataset import CustomImageDataset
from InfereHuggingFace import InfereHF
img_dir_val = '../../../DeepSewerData/DeepSewer/original_data/images_val/'
annotations_file_val = '../../../DeepSewerData/DeepSewer/original_data/SewerML_Aggregated_Val.csv'
input_labels = ['FO','FS','IN','IS','OB','RB','RO','DE','connections','deposits']
valid_dataset = CustomImageDataset(annotations_file_val, img_dir_val, input_labels)
sewerml_infere_resnet = InfereHF('trainedModels/resnetAggregatedClasses',valid_dataset,input_labels)
sewerml_infere_vit = InfereHF('trainedModels/vitAggregatedClasses',valid_dataset,input_labels)

## Infere One Image

In [ ]:
id = 2123
img_test = valid_dataset[id]['image']
gt, pred_resnet = sewerml_infere_resnet.predicSingleImage(id)
gt, pred_vit = sewerml_infere_vit.predicSingleImage(id)
print('groundtruth')
print(gt)
print('resnet')
print(pred_resnet)
print('vit')
print(pred_vit)
display(img_test)

## Infere Set

In [ ]:
ground_truth, predictions_resnet = sewerml_infere_resnet.predictTestDataset()
_, predictions_vit = sewerml_infere_vit.predictTestDataset()
predictions_resnet.to_csv('SewerML_Aggregated_PredictionsResnet.csv',index=False)
predictions_vit.to_csv('SewerML_Aggregated_PredictionsVit.csv',index=False)
ground_truth.to_csv('SewerML_Aggregated_GroundTruth.csv',index=False)

# Evaluation

In [ ]:
from evaluation_models import EvaluationSewerModels
import numpy as np
import pandas as pd
input_labels = ['FO','FS','IN','IS','OB','RB','RO','DE','connections','deposits']
eval_vit = EvaluationSewerModels('SewerML_Aggregated_GroundTruth.csv','SewerML_Aggregated_PredictionsVit.csv',input_labels)
eval_resnet = EvaluationSewerModels('SewerML_Aggregated_GroundTruth.csv','SewerML_Aggregated_PredictionsResnet.csv',input_labels)
metric = 'f2'
score_vit = eval_vit.evaluate_thresholds(metric,0.04)
score_resnet = eval_resnet.evaluate_thresholds(metric,0.04)
print(input_labels)
print(metric)
print('vit',score_vit)
print('resnet',score_resnet)


# Infere external dataset

In [1]:
from InfereHuggingFace import InfereHF

input_labels = ['FO','FS','IN','IS','OB','RB','RO','DE','connections','deposits']
data_set = 'test_imgs'
infere_vit_external_data = InfereHF('trainedModels/vitAggregatedClasses',data_set,input_labels)
results_external_data = infere_vit_external_data.predictSetImgs()
print(results_external_data)
results_external_data.to_csv('test_results.csv',index=False)

/home/deep-sewer/.conda/envs/huggingface/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
100%|██████████| 5/5 [00:00<00:00, 10.75it/s]

                                            filename        FO        FS IN  \
0  W20_P1_f3a4770a-49a9-44e9-83d8-dfadc3bb6e40_20...         0  0.213244  0   
0  W20_P1_c1dd06ba-0c88-463c-b34e-85d9a974f251_20...  0.037451  0.171019  0   
0  W20_P1_7be7d15b-ad4e-4bb5-8920-d55a0549e4c7_20...  0.063941  0.064567  0   
0  W20_P1_1fee1058-07f2-471a-809d-8aa7ebeea64c_20...         0  0.180847  0   
0  W20_P1_e5819c6d-36bc-4135-8ba0-50d264e5ff7d_20...         0  0.191793  0   

  IS        OB        RB RO        DE  connections  deposits  
0  0         0  0.023491  0  0.140288     0.536786  0.041936  
0  0         0  0.000000  0  0.156176     0.522288  0.056571  
0  0         0  0.032435  0  0.121729     0.666132  0.000000  
0  0         0  0.008876  0  0.126041     0.645029  0.011992  
0  0  0.107485  0.064787  0  0.000000     0.232383  0.379093  
